# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
csv_path = "weather_data.csv"
weather_df = pd.read_csv(csv_path)
weather_df

,City,Lat,Lon,Max Temp,Humidity,Wind Speed,Country,Date,Clouds
0,saint-philippe,-21.36,55.77,77.00,78,4.70,RE,1582766425,92
1,hobart,-42.88,147.33,62.60,48,24.16,AU,1582766418,75
2,juba,4.85,31.58,83.93,24,4.99,SS,1582766425,1
3,dikson,73.51,80.55,14.25,89,24.14,RU,1582766425,100
4,barrow,71.29,-156.79,-27.40,74,3.36,US,1582766425,1
...,...,...,...,...,...,...,...,...,...
571,alugan,12.22,125.48,78.85,84,15.97,PH,1582766553,88
572,solnechnyy,50.72,136.63,17.13,80,0.69,RU,1582766553,20
573,panalingaan,8.78,117.42,79.32,76,6.98,PH,1582766553,94
574,akdepe,42.06,59.38,26.60,73,4.70,TM,1582766553,48


In [31]:
maxh=weather_df['Humidity'].max()
print(maxh)

100


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
fig=gmaps.figure()

In [32]:
locations = weather_df[["Lat","Lon"]].astype(float)
humidity_map = weather_df["Humidity"].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_map, dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
# weather_df3 = weather_df[weather_df["Max Temp"]<=80]
# weather_df3 = weather_df[weather_df["Max Temp"]>=70]
# weather_df3 = weather_df[weather_df["Humidity"]<=30]
# weather_df3 = weather_df[weather_df["Clouds"]<=0]
# weather_df3.dropna()
weather_df2 = weather_df[weather_df['Clouds']==0]
weather_df2 = weather_df2[weather_df2['Humidity']<=60]
weather_df2 = weather_df2[weather_df2['Max Temp']<=85]
weather_df2 = weather_df2[weather_df2['Max Temp']>=70]
weather_df2

,City,Lat,Lon,Max Temp,Humidity,Wind Speed,Country,Date,Clouds
86,santa fe,-31.63,-60.70,75.00,53,6.26,AR,1582766444,0
119,savelugu,9.62,-0.83,82.69,14,5.55,GH,1582766454,0
143,dosso,13.05,3.19,73.81,13,9.82,NE,1582766459,0
231,sao filipe,14.90,-24.50,76.82,52,4.25,CV,1582766478,0
255,sur,22.57,59.53,73.22,56,4.97,OM,1582766483,0
303,praia,14.92,-23.51,78.80,36,13.87,CV,1582766493,0
318,abu dhabi,24.47,54.37,71.01,56,12.75,AE,1582766412,0
386,saint-louis,16.33,-15.00,82.40,18,2.24,SN,1582766513,0
443,san juan,-31.54,-68.54,82.58,21,10.18,AR,1582766525,0
487,tecoanapa,16.52,-98.75,81.43,59,5.77,MX,1582766537,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
hotel_df =[]
hotel_lat =[]
hotel_lng =[]
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in weather_df2.iterrows():
    lat = row["Lat"]
    lng = row["Lon"]
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
   
    response = requests.get(base_url, params=params)
    response = response.json()
    results = response["results"]
    
    try:
        hotel_df.append(results[0]["name"])
        hotel_lat.append(results[0]['geometry']["location"]['lat'])
        hotel_lng.append(results[0]['geometry']["location"]['lng'])
    except (KeyError,IndexError):
        hotel_df.append("N/A")
        hotel_lat.append("N/A")
        hotel_lng.append("N/A")

In [34]:
hotel_df

['InterTower Hotel',
 'Nii Dentis Hotel',
 'Galaxy Dosso',
 'Hotel Xaguate',
 'Sur Grand Hotel',
 'Hotel Oasis Atlantico Praiamar',
 'Jumeirah At Etihad Towers',
 'N/A',
 'Hotel Viñas del Sol',
 'Hotel Garapacho']

In [37]:
weather_df2['Hotel Name'] = hotel_df
weather_df2['Hotel Lat'] = hotel_lat
weather_df2['Hotel Lon'] = hotel_lng
weather_df2.dropna()

,City,Lat,Lon,Max Temp,Humidity,Wind Speed,Country,Date,Clouds,Hotel Name,Hotel Lat,Hotel Lon
86,santa fe,-31.63,-60.70,75.00,53,6.26,AR,1582766444,0,InterTower Hotel,-31.6434,-60.7072
119,savelugu,9.62,-0.83,82.69,14,5.55,GH,1582766454,0,Nii Dentis Hotel,9.62871,-0.827556
143,dosso,13.05,3.19,73.81,13,9.82,NE,1582766459,0,Galaxy Dosso,13.0436,3.20888
231,sao filipe,14.90,-24.50,76.82,52,4.25,CV,1582766478,0,Hotel Xaguate,14.8996,-24.4982
255,sur,22.57,59.53,73.22,56,4.97,OM,1582766483,0,Sur Grand Hotel,22.6011,59.481
303,praia,14.92,-23.51,78.80,36,13.87,CV,1582766493,0,Hotel Oasis Atlantico Praiamar,14.9029,-23.5128
318,abu dhabi,24.47,54.37,71.01,56,12.75,AE,1582766412,0,Jumeirah At Etihad Towers,24.4588,54.3217
386,saint-louis,16.33,-15.00,82.40,18,2.24,SN,1582766513,0,N/A,N/A,N/A
443,san juan,-31.54,-68.54,82.58,21,10.18,AR,1582766525,0,Hotel Viñas del Sol,-31.5461,-68.4881
487,tecoanapa,16.52,-98.75,81.43,59,5.77,MX,1582766537,0,Hotel Garapacho,16.5556,-98.7883


In [41]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in weather_df2.iterrows()]
locations = weather_df2[["Lat", "Lon"]]

In [42]:
# Add marker layer ontop of heat map
markers_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))